In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator


appName = "Recommender system "
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Movielens data consist of two data which are ratings and movies.
In fallowing code lines, I have merged two data by movieId column and got one data:

In [ ]:
ratings = spark.read.csv('ratings.csv', inferSchema=True, header=True)
ratings.show(2)
movies = spark.read.csv('movies.csv', inferSchema=True, header=True)
movies.show(2)
ratings=ratings.join(movies,"movieId")
ratings.select("title").show(2)

In [ ]:
data = ratings.select("userId", "movieId","title", "rating")
data.show(2)
(train, test) = data.randomSplit([0.7, 0.3])

In here, I defined our recommendation system algorithm as a ALS function, maximum number of iteration of the model is 10, reguralization paramater of the model is 0.01 then, fit the model.

In [ ]:
als = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop")
model = als.fit(train)

In here, I tested our model and got predictions by test data

In [ ]:
print("Testing:")
prediction = model.transform(test)

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(prediction)
print("RMSE = ",rmse)

I trained our model and checked the accuracy of the model.So, we can get movie recommendation by giving spesific user from our model.
We can see the movie recommendation for user 46.

In [ ]:
users = ratings.select("userId").filter(ratings.userId==46)
userSubsetRecs = model.recommendForUserSubset(users, 1)
userSubsetRecs.show(1)

Movie 501 recommended for user 46 with predicted rating 7.43. Also, We can  learn title of the recommended movie.

In [ ]:
data.select("title","movieId").filter(data.movieId==501).show(1)